In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hw2_1 import *
from torch.optim import Adam
from torch.autograd import Variable
from tqdm.notebook import tqdm
from torchvision.utils import save_image
import torchvision.utils as vutils
import sys
import subprocess
import random

In [3]:
batch_size = 256
lr = 0.0002
b1 = 0.5
b2 = 0.99
epochs = 500
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
test_noise = torch.randn(1000, 100, 1, 1, device='cuda')
fixed_noise = torch.randn(64, 100, 1, 1, device='cuda')

criterion = nn.BCELoss()
generator = Generator().cuda()
discriminator = Discriminator().cuda()

generator.apply(weights_init)
discriminator.apply(weights_init)

dataloader = loadData('hw2_data/face/train/',batch_size,12)
optimizer_G = torch.optim.Adam(generator.parameters(),lr=lr,betas=(b1,b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(),lr=lr,betas=(b1,b2))

In [4]:
def val():
    generator.eval().cuda()
    with torch.no_grad():
        test_noise = torch.randn(1000,100,1,1,device='cuda')
        test_img = generator(test_noise).detach().cpu()
    
    for i in range(1000):
        save_image(test_img[i],'p1_result/'+str(i)+'.png')
    
    FID = subprocess.run([sys.executable,'-m','pytorch_fid','p1_result/','hw2_data/face/val','--device', 'cuda:0'],capture_output=True)
    FACE = subprocess.run([sys.executable,'face_recog.py','--image_dir','p1_result/'],capture_output=True)
    print(FID.stdout.decode(),end='')
    print(FACE.stdout.decode())

In [5]:
img_list = []
G_loss = []
D_loss = []
iters = 0

for epoch in tqdm(range(epochs)):
    for i ,data in enumerate(dataloader,0):
        data = data.cuda()
        optimizer_D.zero_grad()
        label = torch.full((data.shape[0],), 1.0, dtype=torch.float, device='cuda')
        #label += 0.1 * torch.randn((data.shape[0],),device='cuda')
        output = discriminator(data).view(-1)
        real_loss = criterion(output,label)
        real_loss.backward()
        D_x = output.mean().item()
        
        noise = torch.randn(data.shape[0],100,1,1,device='cuda')
        fake = generator(noise)
        label.fill_(0.0)
        #label += 0.1 * torch.randn((data.shape[0],),device='cuda')
        output = discriminator(fake.detach()).view(-1)
        fake_loss = criterion(output,label)
        fake_loss.backward()
        D_G_z1 = output.mean().item()
        
        d_loss = real_loss + fake_loss
        optimizer_D.step()
        
        optimizer_G.zero_grad()
        label.fill_(1.0)
        output = discriminator(fake).view(-1)
        g_loss = criterion(output,label)
        g_loss.backward()
        
        D_G_z2 = output.mean().item()
        optimizer_G.step()
        
#         if i % 50 == 0:
#             print('[%3d/%3d][%3d/%3d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
#                   % (epoch, epochs, i, len(dataloader),
#                      d_loss.item(), g_loss.item(), D_x, D_G_z1, D_G_z2))
            
        G_loss.append(g_loss.item())
        D_loss.append(d_loss.item())
        
#         if (iters % 500 == 0) or ((epoch == epochs-1) and (i == len(dataloader)-1)):
#             with torch.no_grad():
#                 fake = generator(fixed_noise).detach().cpu()
#             img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
#             save_image(fake.data[:25], "p1_result/%d.png" % iters, nrow=5, normalize=True)
        iters += 1
    if(epoch%10==0):
        print("Epoch:",epoch)
        val()
        

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0
FID:  309.62697996486867
Face recognition Accuracy: 0.000%

Epoch: 10
FID:  146.16184072924014
Face recognition Accuracy: 70.529%

Epoch: 20
FID:  73.27982974106644
Face recognition Accuracy: 79.221%

Epoch: 30
FID:  61.826490553099575
Face recognition Accuracy: 76.823%

Epoch: 40
FID:  61.33405203608709
Face recognition Accuracy: 75.724%

Epoch: 50
FID:  58.22679633396751
Face recognition Accuracy: 73.726%

Epoch: 60
FID:  49.03276535301998
Face recognition Accuracy: 79.720%

Epoch: 70
FID:  46.313287387289165
Face recognition Accuracy: 72.228%

Epoch: 80
FID:  51.622824993863105
Face recognition Accuracy: 78.921%

Epoch: 90
FID:  49.082999859381346
Face recognition Accuracy: 78.921%

Epoch: 100
FID:  45.96276945387129
Face recognition Accuracy: 78.521%

Epoch: 110
FID:  54.71483157492298
Face recognition Accuracy: 74.326%

Epoch: 120
FID:  50.28735067018428
Face recognition Accuracy: 79.021%

Epoch: 130
FID:  48.79463309846187
Face recognition Accuracy: 74.525%

Epoch: 140
F

In [6]:
with torch.no_grad():
    fake = generator(test_noise).detach().cpu()
    img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
    save_image(fake.data[:32], "p1_1.png" , nrow=8, normalize=True)

In [7]:
print(generator)
print(discriminator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)
